<a href="https://colab.research.google.com/github/kellytdunn/Seattle-parks/blob/master/Trail_Usage_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### How Do I Avoid Crowds in Seattle Parks During a Pandemic?

For my first Python project, I was curious how the pandemic was affecting transportation, as I am a transportation planner. I have gone out to exercise every day during the stay at home orders, and have observed many others doing the same, particularly at big city parks, like Myrtle Edwards. The City took notice and closed down some parks temporarily that were getting too crowded to allow for social distancing. I wondered if people were more likely to be in parks these days given that there are so few other places to be. I found a dataset from the Seattle Department of Transportation on bicycle and pedestrian counts from 2014-2020 at a specific point at Myrtle Edwards Park: https://data.seattle.gov/Transportation/Elliott-Bay-Trail-in-Myrtle-Edwards-Park-Bicycle-a/4qej-qvrz. 

*Note: This notebook uses two API tokens that I want to keep private while I share the notebook publicly. I define them here in a cell that i can easily hide before sharing. If you want to run these notebooks you'll need to grab your own token.Tokens for bike/ped data can be found here: https://data.seattle.gov/login.Tokens for NOAA data can be found here: https://www.ncdc.noaa.gov/cdo-web/token *

First, I import libraries:

In [2]:
!pip install sodapy

In [3]:
import sys
!{sys.executable} -m pip install ipywidgets>=7.5
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import time
import datetime
import math
import requests
from sodapy import Socrata

In [4]:
#bike/ped count dataset
client = Socrata("data.seattle.gov",
                  SEA_token)

# returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("4qej-qvrz", limit = 100000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [5]:
#convert the datetime field in the "date" column to something more readable and separate date and time.
results_df['Date'] = pd.to_datetime(results_df['date']).dt.date 
results_df['Time'] = pd.to_datetime(results_df['date']).dt.time 
results_df.head()

,date,eilliott_bay_trail_in_myrtle_edwards_park_total,ped_north,ped_south,bike_north,bike_south,Date,Time
0,2020-06-01T00:00:00.000,5,2,1,1,1,2020-06-01,00:00:00
1,2020-06-01T01:00:00.000,0,0,0,0,0,2020-06-01,01:00:00
2,2020-06-01T02:00:00.000,1,0,0,1,0,2020-06-01,02:00:00
3,2020-06-01T03:00:00.000,1,1,0,0,0,2020-06-01,03:00:00
4,2020-06-01T04:00:00.000,1,0,0,1,0,2020-06-01,04:00:00


In [6]:
#rename columns and add columns to help with data analysis.
clean_df = results_df.rename(columns={"date": "Timestamp_Date", "eilliott_bay_trail_in_myrtle_edwards_park_total": "Total"})
clean_df['Day of Week'] = pd.DatetimeIndex(clean_df['Date']).day_name() # week day name
clean_df['Month'] = pd.DatetimeIndex(clean_df['Date']).month_name() # month name
clean_df['Year'] = pd.DatetimeIndex(clean_df['Timestamp_Date']).year # year name
clean_df.head()


,Timestamp_Date,Total,ped_north,ped_south,bike_north,bike_south,Date,Time,Day of Week,Month,Year
0,2020-06-01T00:00:00.000,5,2,1,1,1,2020-06-01,00:00:00,Monday,June,2020
1,2020-06-01T01:00:00.000,0,0,0,0,0,2020-06-01,01:00:00,Monday,June,2020
2,2020-06-01T02:00:00.000,1,0,0,1,0,2020-06-01,02:00:00,Monday,June,2020
3,2020-06-01T03:00:00.000,1,1,0,0,0,2020-06-01,03:00:00,Monday,June,2020
4,2020-06-01T04:00:00.000,1,0,0,1,0,2020-06-01,04:00:00,Monday,June,2020


In [7]:
APIrecords = clean_df.to_dict('records')

In [8]:
len(results_df)

61368

So ```APIrecords``` is a list of dictionaries. Each dictionary represents one hour's worth of data and thus each dictionary has the same keys and different values. There are over 56,000 records in this dataset as of May 2020. As I play around with different functions, I need a quick way to select a subset of records so that every function doesn't iterate through the whole thing.

I create the below ```testlist``` to give myself a small section of the dataset to try out functions on. 

In [9]:
testlist = APIrecords[:5]
testlist

[{'Date': datetime.date(2020, 6, 1),
  'Day of Week': 'Monday',
  'Month': 'June',
  'Time': datetime.time(0, 0),
  'Timestamp_Date': '2020-06-01T00:00:00.000',
  'Total': '5',
  'Year': 2020,
  'bike_north': '1',
  'bike_south': '1',
  'ped_north': '2',
  'ped_south': '1'},
 {'Date': datetime.date(2020, 6, 1),
  'Day of Week': 'Monday',
  'Month': 'June',
  'Time': datetime.time(1, 0),
  'Timestamp_Date': '2020-06-01T01:00:00.000',
  'Total': '0',
  'Year': 2020,
  'bike_north': '0',
  'bike_south': '0',
  'ped_north': '0',
  'ped_south': '0'},
 {'Date': datetime.date(2020, 6, 1),
  'Day of Week': 'Monday',
  'Month': 'June',
  'Time': datetime.time(2, 0),
  'Timestamp_Date': '2020-06-01T02:00:00.000',
  'Total': '1',
  'Year': 2020,
  'bike_north': '1',
  'bike_south': '0',
  'ped_north': '0',
  'ped_south': '0'},
 {'Date': datetime.date(2020, 6, 1),
  'Day of Week': 'Monday',
  'Month': 'June',
  'Time': datetime.time(3, 0),
  'Timestamp_Date': '2020-06-01T03:00:00.000',
  'Total': 

#### Creating Functions to Prepare the Data

In [10]:
def timestamp(mmddyyyy):
    return pd.to_datetime(mmddyyyy)

In [11]:
def hourlycounts(date, data): #date must be entered as 'mm/dd/yy', including quotes
    target_date = timestamp(date) #this caches the function so it doesn't have to recalculate with every iteration. 
    return [x['Total'] for x in data if x['Date'] == target_date] #this is a cleaner list comprehension. Thank you Stack Overflow.


If I want to be able to create graphs with time or date as an independent variable, I need to be able to generate a list of all the values associated with a specific key in the list of dictionaries. Below, I create a function and test it out on ```testlist```.

In [12]:
def list_from_dictionary(dictlist, key):
    return [sub[key] for sub in dictlist]
list_from_dictionary(testlist, 'Day of Week')

['Monday', 'Monday', 'Monday', 'Monday', 'Monday']

Trying this function out on ```testlist``` confirms that it would not filter out duplicates, and if I run this on the whole dataset it would return 54,000 items, which I don't want. 

To create a list of all the values for a given key, without duplicates, I need a new function. 

In [13]:
#generate list of all unique values in a given column in the dataset. Use to generate x values for graphs and in for loops.
def unique(recordlist, key):
    new_list=[]
    list1 = list_from_dictionary(recordlist, key)
    list_set = set(list1)
    unique_list = list(list_set)
    for x in unique_list:
        new_list.append(x)
    return new_list
unique(testlist, 'Day of Week')

['Monday']

I apply this to the 'Time' key, as I plan to use hour of day as an x-value for my initial graphs.

In [14]:
alltimes = sorted(unique(APIrecords, 'Time'))


Now I want to know how April 2020 overall compares to other years to see if, overall, April 2020 might see different usage than prior years. I'll choose Fridays in April as a way to compare apples to apples across different years.
To start with, the function below would give you, for example, the average count at noon for all the Fridays in April 2020. 

In [15]:
def hourly_average_by_day(month, year, weekday, hour):
    monthly_total = []
    matching_records = list(record for record in APIrecords if record['Day of Week']==weekday and record['Month']==month and record['Time']==hour and record['Year']==year)
    for record in matching_records:
        monthly_total.append(int(record['Total']))
    try: 
        return sum(monthly_total)/len(monthly_total)
    except:
        return 'error'

#hourly_average_by_day('March', 2017, 'Friday', datetime.time(17, 0))


The function above gives me average counts for a specific hour of a specific weekday in the year and month I provide as arguments. Now I want to be able to apply that function to every year all at once, so I can graph it and see a change over time. Maybe people are using the park at different times of day than they used to, and I'm curious how that's changed. The following function applies ```hourly_average_by_day()``` to every year from 2014-2020.

First, though, I noticed when looking through the data that no counts were recorded for a large part of 2015; the equipment must have been broken. I don't want to get zeros in my plots, so I'm going to omit 2015 data. 

In [16]:
def counts_across_years(month, weekday, hour): #note below how the hour must be formatted.
    counts_list=[]
    for year in [x for x in range(2014, 2021) if x != 2015]:
        count = hourly_average_by_day(month, year, weekday, hour)
        counts_list.append(count)
    return counts_list

#counts_across_years('April', 'Friday', datetime.time(12, 0))

In [17]:
allyears=[]
for i in [x for x in range(2014, 2021) if x != 2015]:
    allyears.append(i)
allyears

[2014, 2016, 2017, 2018, 2019, 2020]

#### Graphing Trail Usage Over Time

Now it's time to make my first graph. I will use my ```counts_across_years()``` function to graph how average hourly counts at noon for any given weekday have changed from 2014-2020. I am curious if 2020 will continue past trends or show a departure. 

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Monday', datetime.time(12, 0)),
                    mode='lines',
                    name='Mondays, 12-1 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Tuesday', datetime.time(12, 0)),
                    mode='lines',
                    name='Tuesdays, 12-1 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Wednesday', datetime.time(12, 0)),
                    mode='lines',
                    name='Wednesdays, 12-1 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Thursday', datetime.time(12, 0)),
                    mode='lines',
                    name='Thursdays, 12-1 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Friday', datetime.time(12, 0)),
                    mode='lines',
                    name='Fridays, 12-1 pm'))
fig.update_layout(title='Average Park Activity During Different Days of the Week in April, Lunchtime',
                   xaxis_title='Year',
                   yaxis_title='Average Bike + Ped Count')
fig.show()

This data suggests that park usage on any given weekday in April 2020 is higher compared to the same weekday in 2019, except on Tuesdays, but compared to prior years there's no recognizable pattern. This is just the lunch hour though. Will this pattern hold after work hours?

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Monday', datetime.time(17, 0)),
                    mode='lines',
                    name='Mondays, 5-6 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Tuesday', datetime.time(17, 0)),
                    mode='lines',
                    name='Tuesdays, 5-6 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Wednesday', datetime.time(17, 0)),
                    mode='lines',
                    name='Wednesdays, 5-6 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Thursday', datetime.time(17, 0)),
                    mode='lines',
                    name='Thursdays, 5-6 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Friday', datetime.time(17, 0)),
                    mode='lines',
                    name='Fridays, 5-6 pm'))
fig.add_trace(go.Scatter(x=allyears, y=counts_across_years('April', 'Saturday', datetime.time(17, 0)),
                    mode='lines',
                    name='Saturdays, 5-6 pm'))

fig.update_layout(title='Average Park Activity During Different Days of the Week in March, Evenings',
                   xaxis_title='Year',
                   yaxis_title='Average Bike + Ped Count')
fig.show()

Interestingly, the same pattern doesn't hold from the 5 to 6 pm hour, at least not every day. While Tuesday and Wednesday afternoons see a steep drop from 2019, Thursday and Fridays see increases. But while in 2019 there was  noticeably higher usage earlier in the week, in 2020 usage is converging on all days except Monday. Perhaps the pandemic has wiped away most of the weekly rhythms that distinguish one weekday from another. 

Now I'd like to get a sense of how usage varies throughout the day. This could help people determine the least crowded time to visit the park, for example. The function below takes a month, year, and weekday (for example, Fridays in March 2020) and returns average counts for each hour: average counts at 10am, average counts at 11am, etc.

In [20]:
def monthly_average_by_hour(month, year, weekday):
    hourly_average=[]
    hourlist = sorted(unique(APIrecords, 'Time'))
    for hour in hourlist:
        hourly_average.append(hourly_average_by_day(month, year, weekday, hour))
    return hourly_average
monthly_average_by_hour('April', 2020, 'Friday')
#it would be best if this were a dictionary so we could ensure keys and values are matched up.

[12.25,
 4.0,
 4.75,
 2.25,
 8.75,
 20.0,
 112.75,
 204.0,
 188.75,
 210.5,
 241.0,
 282.75,
 334.5,
 371.0,
 350.0,
 403.75,
 436.75,
 501.0,
 529.0,
 342.75,
 128.0,
 40.5,
 29.5,
 10.0]

That data looks about right, based on what I've observed on various of my own walks! Usage peaks in late afternoon when people get off work and it is warmer. For now I'm going to apply this function to different years in the dataset to compare trends year over year. But instead of Friday, I'll graph Wednesday, because in transportation terms you get a better picture of people's habits if you choose data in the middle of the week. 

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=alltimes, y=monthly_average_by_hour('April', 2020, 'Wednesday'),
                    mode='lines',
                    name='2020'))
fig.add_trace(go.Scatter(x=alltimes, y=monthly_average_by_hour('April', 2019, 'Wednesday'),
                    mode='lines',
                    name='2019'))
fig.add_trace(go.Scatter(x=alltimes, y=monthly_average_by_hour('April', 2018, 'Wednesday'),
                    mode='lines',
                    name='2018'))
fig.add_trace(go.Scatter(x=alltimes, y=monthly_average_by_hour('April', 2017, 'Wednesday'),
                    mode='lines',
                    name='2017'))
fig.add_trace(go.Scatter(x=alltimes, y=monthly_average_by_hour('April', 2016, 'Wednesday'),
                    mode='lines',
                    name='2016'))
fig.update_layout(title='Average Hourly Park Activity for Wednesdays in April',
                   xaxis_title='Hour of Day',
                   yaxis_title='Average Bike + Ped Count')
fig.show()

So activity in April 2020 does not really stand out compared to other years.

So far we've been looking at April 2020 only as a whole. Let's zoom in on April 2020 by graphing hourly counts for every Friday that month. I posit that counts will creep up week after week, as more and more people began working from home and looking for ways to get some exercise during and after work.

In [22]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=alltimes, y=hourlycounts('4/3/2020', APIrecords),
                    mode='lines',
                    name='April 3',
                    line_color='rgb(40, 246, 212)'),
                    secondary_y=False)
fig.add_trace(go.Scatter(x=alltimes, y=hourlycounts('4/10/2020', APIrecords),
                    mode='lines',
                    name='April 10',
                    line_color='rgb(27, 164, 161)'),
                    secondary_y=False)
fig.add_trace(go.Scatter(x=alltimes, y=hourlycounts('4/17/2020', APIrecords),
                    mode='lines',
                    name='April 17',
                    line_color='rgb(15, 82, 110)'),
                    secondary_y=False)
fig.add_trace(go.Scatter(x=alltimes, y=hourlycounts('4/24/2020', APIrecords),
                    mode='lines',
                    name='April 24',
                    line_color='rgb(3, 1, 60)'),
                    secondary_y=False)

fig.update_layout(title='Bike and Ped Activity in Myrtle Edwards Park - Fridays During Covid-19 (2020)',
                   xaxis_title='Hour of Day',
                   yaxis_title='Total Bike + Ped Count')
fig.show()

Well, I was wrong. Bicyle and pedestrian counts seem to vary quite a bit by day, and don't seem to be increasing over time. Could weather be a factor in day to day differences? Let's dig into some more data to find out.

#### Investigating Weather Factors

In [23]:
#x axis: weeks or days
#y axis: total count of the week
alldays = sorted(unique(APIrecords, 'Date')) #what is this for? not sure if sorting will matter once we have a df.

##### Accessing Weather Data

I will use the Climate Data Online API from NOAA: https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted. It includes a dataset on historical high temperatures by day. 

I will also use the cdo-api-py library for this, which is a huge help. https://pypi.org/project/cdo-api-py/. With this library I don't have to worry about creating separate queries to get around request limits, or converting Celsius to Fahrenheit. 

In [24]:
!pip install cdo_api_py
!pip install config

In [25]:
from cdo_api_py import Client

# Utils
import config
import pandas as pd
from datetime import datetime
from pprint import pprint

# Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [26]:
# Define the client

my_client = Client(NOAA_token, default_units='standard', default_limit=1000)

######################################
#           Configuration            #
######################################

# Select an area
extent = {
    "north": 47.9,
    "south": 47.3,
    "east": -122.26,
    "west": -122.68,
}

# Selected Dates
startdate = datetime(2014, 1, 1)
enddate = datetime(2020, 12, 31)

# Datasets we will query
datasetid='GHCND'


# PRCP = Precipitation (mm or inches as per user preference, inches to hundredths on Daily Form pdf file)
# SNOW = Snowfall (mm or inches as per user preference, inches to tenths on Daily Form pdf file)
# TMAX = Maximum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on
# Daily From pdf file

# Data Types
datatypeid=['TMAX', 'PRCP', 'SNOW']

In [27]:
# Find weather stations with data
stations = my_client.find_stations(
    datasetid=datasetid,
    extent=extent,
    startdate=startdate,
    enddate=enddate,
    datatypeid=datatypeid,
    return_dataframe=True)


# Create an empty dataframe to store climate data
climate_data = pd.DataFrame()

for rowid, station in stations.iterrows():  
    station_data = my_client.get_data_by_station(
        datasetid=datasetid,
        stationid=station['id'],
        startdate=startdate,
        enddate=enddate,
        return_dataframe=True,
        include_station_meta=True   # flatten station metadata with ghcnd readings
    )
    climate_data = pd.concat([climate_data, station_data])

https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=GHCND&startdate=2014-01-01&enddate=2020-12-31&extent=47.3&extent=-122.68&extent=47.9&extent=-122.26&datatypeid=TMAX&datatypeid=PRCP&datatypeid=SNOW&limit=1000&units=standard
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations/GHCND:USW00024233?limit=1000&units=standard
https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2014-01-01&enddate=2015-01-01&stationid=GHCND:USW00024233&limit=1000&units=standard
https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2014-01-01&enddate=2015-01-01&stationid=GHCND:USW00024233&limit=1000&units=standard&offset=1000
https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2014-01-01&enddate=2015-01-01&stationid=GHCND:USW00024233&limit=1000&units=standard&offset=2000
https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2014-01-01&enddate=2015-01-01&stationid=GHCND:USW00024233&limit=1000&units=standard&offset=3000
https://www.

In [28]:
climate_data.head()

,station,date,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT08,WT09,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude
0,GHCND:USW00024233,2014-01-01T00:00:00,2.7,NaN,0.00,0.0,0.0,42.0,45.0,38.0,340.0,310.0,8.1,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.8,1948-01-01,2021-02-19,47.4444,"SEATTLE TACOMA AIRPORT, WA US",1,METERS,-122.3138
1,GHCND:USW00024233,2014-01-02T00:00:00,7.2,NaN,0.16,0.0,0.0,45.0,51.0,43.0,190.0,200.0,21.0,25.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.8,1948-01-01,2021-02-19,47.4444,"SEATTLE TACOMA AIRPORT, WA US",1,METERS,-122.3138
2,GHCND:USW00024233,2014-01-03T00:00:00,5.8,NaN,0.06,0.0,0.0,45.0,48.0,37.0,30.0,50.0,14.1,16.1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,112.8,1948-01-01,2021-02-19,47.4444,"SEATTLE TACOMA AIRPORT, WA US",1,METERS,-122.3138
3,GHCND:USW00024233,2014-01-04T00:00:00,6.0,NaN,0.00,0.0,0.0,41.0,46.0,33.0,40.0,40.0,10.1,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,112.8,1948-01-01,2021-02-19,47.4444,"SEATTLE TACOMA AIRPORT, WA US",1,METERS,-122.3138
4,GHCND:USW00024233,2014-01-05T00:00:00,8.3,NaN,0.00,0.0,0.0,37.0,47.0,31.0,10.0,10.0,15.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.8,1948-01-01,2021-02-19,47.4444,"SEATTLE TACOMA AIRPORT, WA US",1,METERS,-122.3138


##### Preparing Count Data for a Regression

First I want take the list of dictionaries `APIrecords` and return a single dictionary with date: count as the key: value pair. However, currently each record represents an hour, so I will need to aggregate these into total daily counts first using a new ``dailysum`` function I create below. 

for reference:

def hourlycounts(date): #date must be entered as 'mm/dd/yy', including quotes
    y_values=[]
    date_filtered = list(record for record in APIrecords if record['Date']==timestamp(date)) #checks for records that match the input date.
    list(map(lambda record: y_values.append(record['Total']), date_filtered))
    return y_values
    

In [29]:
def dailysum(date):
    strlist = hourlycounts(date, APIrecords) #produces a list of strings
    try: 
        intlist = list(map(int, strlist)) #converts list of strings to integers
        return sum(intlist) #sums the integers
    except ValueError: #when data is NaN, give a 0 for that day, but it shouldn't affect other days
        return 0

In [30]:
dailysum('03/20/2020')

6028

Next I will utilize the `unique` function I created earlier to generate a unique list of all dates in the dataset which I will then plug into the next function.

In [31]:
alldates = sorted(unique(APIrecords, 'Date')) 
testdates = alldates[2300:2305] 

In [32]:
testdates

[datetime.date(2020, 4, 19),
 datetime.date(2020, 4, 20),
 datetime.date(2020, 4, 21),
 datetime.date(2020, 4, 22),
 datetime.date(2020, 4, 23)]

Finally I create `dict_of_counts` to get my dictionary with date:count as the key:value pair. I test it with the `testdates` list I created above, which is just a subset of the `alldates` list. `alldates` is over 2000 dates so it takes a long time to run.

In [33]:
def dict_of_counts(datelist):
    master_dict = {}
    for date in datelist:
        master_dict.update({date: (dailysum(date))})
        #master_dict[date] = dailysum(date)
    return master_dict

print(dict_of_counts(testdates))

{datetime.date(2020, 4, 19): 8319, datetime.date(2020, 4, 20): 6920, datetime.date(2020, 4, 21): 3752, datetime.date(2020, 4, 22): 1754, datetime.date(2020, 4, 23): 3299}


##### Preparing Weather Data for a Regression

OK, that is the counts dictionary. Now I want to work on simplifying the weather dataframe.

def dict_of_temps(records): #takes a list of dictionaries from NOAA dataset and returns a single dictionary with Date: Temperature as key: value. only one year at a time.
    master_dict = dict()
    for record in records: #record would be a dictionary containing one date with one temp reading if `records` is a list of dictionaries. 
        master_dict.update({record['Date']: convert_temp(record['value'])})
    return master_dict


Now I need to create a new dataframe with temperature and count so I can associate each one with the same date and therefore ensure that the temperature and count are matched properly when I do a regression.

##next step:
Create a series from each dataframe

In [34]:
date_list = alldates

In [35]:
counts_series = pd.Series(dict_of_counts(date_list))

In [36]:
counts_series.index = dict_of_counts(date_list).keys()

In [37]:
type(counts_series)

pandas.core.series.Series

In [38]:
counts_series.head()

2014-01-01    2190
2014-01-02    1580
2014-01-03    2469
2014-01-04    3337
2014-01-05    2849
dtype: int64

In [39]:
temp_series = climate_data.iloc[:,8]
temp_series.head()


0    45.0
1    51.0
2    48.0
3    46.0
4    47.0
Name: TMAX, dtype: float64

In [40]:
temp_series.index = pd.to_datetime(climate_data.iloc[:,1]).dt.date #takes column 1 from the climate_data df and uses it as the index here. At the same time, converts the date to the same format as temp_series. 
temp_series.head()

date
2014-01-01    45.0
2014-01-02    51.0
2014-01-03    48.0
2014-01-04    46.0
2014-01-05    47.0
Name: TMAX, dtype: float64

In [41]:
df_dict = {'count':counts_series, 'temperature':temp_series}
df = pd.DataFrame(df_dict)
df.head()

,count,temperature
date,,
2014-01-01,2190,45.0
2014-01-02,1580,51.0
2014-01-03,2469,48.0
2014-01-04,3337,46.0
2014-01-05,2849,47.0


In [42]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['temperature'], y=df['count'],
                    mode='markers',
                    name=''))
fig.update_layout(title='Trail Usage as a Function of Temperature',
                   xaxis_title='High Temperature (F)',
                   yaxis_title='Daily Trail Usage')
fig.show()


In [43]:
#next steps: color 2020 differently; add post-covid 2020 as a dummy variable; incorporate other variables